In [31]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

# local modules 
import sys
sys.path.insert(0, "../scripts")
import helpers as h
import seaborn as sns

import matplotlib.pyplot as plt

from numpy.linalg import lstsq

In [3]:
a00, a01 = h.import_desired_data("A", "15T")
# TODO edit in function 
a00.set_index(a00["DateTime"].values, inplace=True)
a01.set_index(a01["DateTime"].values, inplace=True)
print(a00["Window Open"].unique(), a01["Window Open"].unique(), a00.columns)

temp = a01["Temp C"]
time = a01["DateTime"]

[1.] [0. 1.] Index(['DateTime', 'Temp C', 'RH %', 'Room', 'Ambient Temp', 'Ambient RH',
       'Window Open'],
      dtype='object')


In [4]:
def make_fit(series, A, w, phi, w_t_var):
    n = len(series)
    t = np.arange(n)

    w_t = np.random.normal(0, 1, size=w_t_var)
    x_t = A*np.cos(2*np.pi*w*t + phi) # + w_t

    return x_t, t

In [25]:
def norm_grad(series):
    return h.normalize(np.gradient(series))

In [30]:
x_t, t = make_fit(series=temp, A=-0.5, w=3.9*(1/(len(temp))), phi=0.8, w_t_var=1)
x_t += 0.5

pi = np.pi
x_t2, t2 = make_fit(series=temp, A=-0.25, w=3.9*(1/(len(temp))), 
    phi=-2.3, w_t_var=1)
x_t2 += 0.3

sum_xt = x_t + x_t2

norm_temp = h.normalize(temp)
dif_xt = norm_temp - x_t 
ddt_dif_xt= h.normalize(np.gradient(dif_xt))

dif_sum = sum_xt - norm_temp
ddt_dif_sum = h.normalize(np.gradient(dif_sum))

amb = h.normalize(a00["Ambient Temp"])
ddt_amb = norm_grad(amb)
ddt2_amb = norm_grad(ddt_amb)
smooth_amb = amb.ewm(com=8).mean()
ddt_smooth_amb = h.normalize(np.gradient(smooth_amb))


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time,
    y=a01["Window Open"],
    name="Window",
    mode='lines',
))


# fig.add_trace(go.Scatter(
#     x=time,
#     y=norm_temp,
#     mode='lines',
#     name="Var Temp, Norm"
# ))
# fig.add_trace(go.Scatter(
#     x=time,
#     y=x_t,
#     name="x_t",
#     mode='lines',
# ))

# fig.add_trace(go.Scatter(
#     x=time,
#     y=dif_xt,
#     name="dif_xt",
#     mode='lines',
# ))


fig.add_trace(go.Scatter(
    x=time,
    y=ddt_dif_xt,
    name="ddt_dif_xt",
    mode='lines',
))

fig.add_trace(go.Scatter(
    x=a01["DateTime"],
    y=amb, 
    name="Ambient",
    mode='lines',
))

fig.add_trace(go.Scatter(
    x=a01["DateTime"],
    y=ddt_amb, 
    name="ddt_Ambient",
    mode='lines',
))

fig.add_trace(go.Scatter(
    x=a01["DateTime"],
    y=ddt2_amb, 
    name="ddt_Ambient",
    mode='lines',
))


# fig.add_trace(go.Scatter(
#     x=a01["DateTime"],
#     y=smooth_amb, 
#     name="Smooth Ambient",
#     mode='lines',
# ))


# fig.add_trace(go.Scatter(
#     x=a01["DateTime"],
#     y=ddt_smooth_amb, 
#     name="Ddt Smooth Ambient",
#     mode='lines',
# ))



In [52]:
fig = go.Figure()

n_arr = np.arange(len(amb)) 

# close examine 
xsol, ysol = SegmentedLinearReg(n_arr, amb, [41, 67, 100, 135, 192, 231, 281, 330, 364])

# generatlize 
peak_times = np.arange(4*2)
a = 30 + (100/2)*peak_times
a = np.append(a, [(a[1] + a[2])/2])
print(a)

xsol2, ysol2 = SegmentedLinearReg(n_arr, amb, a)

fig.add_trace(go.Scatter(
    x=n_arr,
    y=amb,
    name="Window",
    mode='lines',
))

fig.add_trace(go.Scatter( x=n_arr, y=amb, name="Ambient", mode='lines',
))

fig.add_trace(go.Scatter(x=xsol, y=ysol, name="Close Fit", mode='lines',
))

fig.add_trace(go.Scatter( x=xsol2, y=ysol2, name="Generalizable Fit", mode='lines', ))

maximum iteration reached
[ 30.  80. 130. 180. 230. 280. 330. 380. 105.]
maximum iteration reached


array([ 30.,  80., 130., 180., 230., 280., 330., 380.])

In [39]:
ysol

array([ 0.21095767,  0.81736061,  0.20413028,  0.2577005 ,  0.80132228,
       -0.04444522,  1.0178313 , -0.05705681,  0.76279699,  0.10030266,
        0.2757114 ])

In [32]:
ramp = lambda u: np.maximum( u, 0 )
step = lambda u: ( u > 0 ).astype(float)

def SegmentedLinearReg( X, Y, breakpoints ):
    nIterationMax = 10

    breakpoints = np.sort( np.array(breakpoints) )

    dt = np.min( np.diff(X) )
    ones = np.ones_like(X)

    for i in range( nIterationMax ):
        # Linear regression:  solve A*p = Y
        Rk = [ramp( X - xk ) for xk in breakpoints ]
        Sk = [step( X - xk ) for xk in breakpoints ]
        A = np.array([ ones, X ] + Rk + Sk )
        p =  lstsq(A.transpose(), Y, rcond=None)[0] 

        # Parameters identification:
        a, b = p[0:2]
        ck = p[ 2:2+len(breakpoints) ]
        dk = p[ 2+len(breakpoints): ]

        # Estimation of the next break-points:
        newBreakpoints = breakpoints - dk/ck 

        # Stop condition
        if np.max(np.abs(newBreakpoints - breakpoints)) < dt/5:
            break

        breakpoints = newBreakpoints
    else:
        print( 'maximum iteration reached' )

    # Compute the final segmented fit:
    Xsolution = np.insert( np.append( breakpoints, max(X) ), 0, min(X) )
    ones =  np.ones_like(Xsolution) 
    Rk = [ c*ramp( Xsolution - x0 ) for x0, c in zip(breakpoints, ck) ]

    Ysolution = a*ones + b*Xsolution + np.sum( Rk, axis=0 )

    return Xsolution, Ysolution

In [10]:
amb = h.normalize(a00["Ambient Temp"])

# try piecewise linear fit??

In [12]:
amb.rolling(2).mean()

2022-07-20 07:15:00         NaN
2022-07-20 07:30:00    0.222591
2022-07-20 07:45:00    0.222591
2022-07-20 08:00:00    0.232558
2022-07-20 08:15:00    0.242525
                         ...   
2022-07-24 06:15:00    0.239203
2022-07-24 06:30:00    0.239203
2022-07-24 06:45:00    0.239203
2022-07-24 07:00:00    0.242525
2022-07-24 07:15:00    0.245847
Name: Ambient Temp, Length: 385, dtype: float64

In [9]:
# moving average filter on ambient data vt = theta(w_t-1 + w_t + w_t+1 )
amb = h.normalize(a00["Ambient Temp"])

def moving_avg(ts):
    
    return smooth_ts


2022-07-20 07:15:00    0.222591
2022-07-20 07:30:00    0.222591
2022-07-20 07:45:00    0.222591
2022-07-20 08:00:00    0.242525
2022-07-20 08:15:00    0.242525
                         ...   
2022-07-24 06:15:00    0.239203
2022-07-24 06:30:00    0.239203
2022-07-24 06:45:00    0.239203
2022-07-24 07:00:00    0.245847
2022-07-24 07:15:00    0.245847
Name: Ambient Temp, Length: 385, dtype: float64